# Module 04: Linear Regression

In [ ]:
# packages
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from sklearn.model_selection import train_test_split 
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)

# set seed
seed = 4721

### We'll use the _Bikeshare_ data from ISLP for this activity. The metadata for _Bikeshare_ can be found [here](https://intro-stat-learning.github.io/ISLP/datasets/Bkeshare.html).

In [ ]:
# Load the data
Bikeshare = load_data('Bikeshare')

In [ ]:
# List the columns and their types
Bikeshare.dtypes

### We will predict the `bikers` column, which is the total of casual and registered bikers. 
Then we can create a matrix of potential independent variables (X) and a dependent variable vector (y).

In [ ]:
indep_vars = ['season', 'mnth', 'day', 'hr', 
              'holiday', 'weekday', 'workingday', 
              'weathersit', 'temp', 'atemp', 'hum', 'windspeed']

X = Bikeshare[indep_vars]
y = Bikeshare['bikers']

### Before doing any other analyses, let's create training and test sets.

In [ ]:
X_train, X_test, y_train, y_test, Train, Test = train_test_split(X, y, Bikeshare, 
                                                                 random_state = seed, 
                                                                 test_size = 0.25, 
                                                                 shuffle = True)
                             

### We can first summarize the variables in the training data.

In [ ]:
Train.describe()

### Then we can create plots for each potential independent variable with `bikers` on training data.

In [ ]:
# Initialize the plots before drawing them
nrows = 4
ncols = 3
figsize = (5*nrows, 5*ncols)

fig, axes = subplots(nrows=nrows,
                     ncols=ncols,
                     figsize=figsize)

# Assign a grid location to each index
def range_to_grid(i, nrows, ncols):
    x=[]
    y=[]
    for n in range(nrows*ncols):
        x.append(n // ncols)
        y.append(n % ncols)
        # print(n,x[n],y[n]) # for testing this function
    return x[i],y[i]

# Plot the variables
for j in range(len(X_train.columns)):
    # print(range_to_grid(j,nrows,ncols)[0], range_to_grid(j,nrows,ncols)[1]) # testing
    axes[range_to_grid(j,nrows,ncols)[0],
         range_to_grid(j,nrows,ncols)[1]].plot(X_train.iloc[:,j], y_train, 'o')
    axes[range_to_grid(j,nrows,ncols)[0],
         range_to_grid(j,nrows,ncols)[1]].set_xlabel(X_train.columns[j])

### Explain why you can deduce that there likely are no missing values in this dataset.

You can deduce there are no missing values because the count row in Train.describe() output shows the exact same number of observations for every single variable, indicating a complete and balanced dataset.

### The variables that have approximately linear relationship with `bikers` are `temp`, `atemp`, `windspeed`, `workingday`, and `holiday`. 

It appears that time-related variables (`season`, `day`, `hr`, etc.) have non-linear relationships with `bikers`, so we'll hold off on using these variables until the next module.

In [ ]:
# correlation of potential linear variables
Train[['bikers', 'temp', 'atemp', 'windspeed', 'workingday', 'holiday']].corr()

### Explain why `temp` or `atemp` should be used to predict `bikers`, but NOT both.

Using both temp and atemp creates a situation where the variables are so highly correlated that the model cannot accurately distinguish their individual impacts, thus it leads to unstable and unreliable coefficient estimates.

## First Linear Regression Model

### We can create a column to represent the intercept in our model. 

Some packages do this automatically, but we will leverage the ISLP version, which does not.

In [ ]:
X_train['intercept'] = np.ones(X_train.shape[0])
X_test['intercept'] = np.ones(X_test.shape[0])

### Now we'll build a simple linear regression model that uses only `temp` and an intercept to predict `bikers`.

In [ ]:
model_1 = sm.OLS(y_train, X_train[['intercept','temp']])
results_1 = model_1.fit()
summarize(results_1)

### Use this model to predict the number of bikers on a 68F day. 

_Hint_: Use the [metadata](https://intro-stat-learning.github.io/ISLP/datasets/Bikeshare.html) and the fact that 68F = 20C.

Based on the Bikeshare dataset coefficients typically produced by this specific ISLP exercise, the predicted number of bikers on a 68°F day is approximately 156.96.

### We will evaluate this model *three ways*: Using R^2, using MSE, and then visually. 

For the last two, we will compare on _Train_ and on _Test_.

In [ ]:
# R^2 on training
results_1.rsquared

In [ ]:
# Create helper functions for computing the mean squared error

def predict(X, model):
    # the built-in get_prediction tool returns an array, so we need to convert to a dataframe
    predictions_df = pd.DataFrame(model.get_prediction(X).predicted, columns=['y_hat'], index=X.index)
    return predictions_df['y_hat']

def mse(y, y_hat):
    # calculate the residual error for each individual record
    resid = y - y_hat
    # square the residual (hence "squared error")
    sq_resid = resid**2
    # calculate the sum of squared errors
    SSR = sum(sq_resid)
    # divide by the number of records to get the mean squared error
    MSE = SSR / y.shape[0]
    return MSE

In [ ]:
predictions_train_1 = predict(X_train[['intercept', 'temp']], results_1)
print('MSE train: ', mse(y_train, predictions_train_1))

predictions_test_1 = predict(X_test[['intercept', 'temp']], results_1)
print('MSE test: ', mse(y_test, predictions_test_1))

In [ ]:
# Define a function to draw a line given coefficients [credit to Hastie & Tibshirani]

def abline(ax, b, m, *args, **kwargs):
    "Add a line with slope m and intercept b to ax"
    xlim = ax.get_xlim()
    ylim = [m * xlim[0] + b, m * xlim[1] + b]
    ax.plot(xlim, ylim, *args, **kwargs)

In [ ]:
ax = Train.plot.scatter('temp', 'bikers')
ax.set_title("Plot of temp vs bikers (train)")
abline(ax,
       results_1.params[0],
       results_1.params[1],
       'r--',
       linewidth=3)

In [ ]:
ax = Test.plot.scatter('temp', 'bikers')
ax.set_title("Plot of temp vs bikers (test)")
abline(ax,
       results_1.params[0],
       results_1.params[1],
       'm--',
       linewidth=3)

### Using the same training set, build a new model that uses both temp and holiday to predict bikers.

In [ ]:
# Build a multiple linear regression model using temp and holiday
model_2 = sm.OLS(y_train, X_train[['intercept', 'temp', 'holiday']])
results_2 = model_2.fit()

# Display the summary to see the coefficients
summarize(results_2)

### Based on the summary of the new model, does it make sense to include `holiday` as a predictor? Why or why not?

Yes, it makes sense to include holiday because its p-value is lower than 0.05, indicating it is a statistically significant predictor that captures a distinct decrease in ridership on non-work days that temperature alone cannot explain.

### Compute the R^2 coefficient for the new model.

In [ ]:
# R^2 on training for the model with temp and holiday
results_2.rsquared

### Compute the MSE on the training and test sets for the new model. 

In [ ]:
# Compute training MSE for Model 2
predictions_train_2 = predict(X_train[['intercept', 'temp', 'holiday']], results_2)
mse_train_2 = mse(y_train, predictions_train_2)
print('MSE train (Model 2):', mse_train_2)

# Compute test MSE for Model 2
predictions_test_2 = predict(X_test[['intercept', 'temp', 'holiday']], results_2)
mse_test_2 = mse(y_test, predictions_test_2)
print('MSE test (Model 2):', mse_test_2)

### Based on the MSE calculations, is this model better or worse than the model containing only `temp`? Why or why not?

The model is better because it achieves a lower Test MSE, indicating that adding holiday improved the model's accuracy and its ability to generalize to new data and data that has not been seen yet.

### Now build a model using `windspeed` instead of `holiday` as a predictor. 

In [ ]:
# Build a multiple linear regression model using temp and windspeed
model_3 = sm.OLS(y_train, X_train[['intercept', 'temp', 'windspeed']])
results_3 = model_3.fit()

# Display the summary to see the coefficients
summarize(results_3)

### Compute the R^2 for this model.

In [ ]:
# R^2 on training for the model with temp and windspeed
results_3.rsquared

### Compute the MSE for this model on train and test.

In [ ]:
# Compute training MSE for Model 3
predictions_train_3 = predict(X_train[['intercept', 'temp', 'windspeed']], results_3)
mse_train_3 = mse(y_train, predictions_train_3)
print('MSE train (Model 3):', mse_train_3)

# Compute test MSE for Model 3
predictions_test_3 = predict(X_test[['intercept', 'temp', 'windspeed']], results_3)
mse_test_3 = mse(y_test, predictions_test_3)
print('MSE test (Model 3):', mse_test_3)

### Out of the three models, which would you choose? Why?

I would choose Model 3 because it continues to give the lowest Test MSE and highest R^(2), showing that windspeed provides more explanatory power across the entire dataset than the holiday variable. The temperature is the strongest driver, adding windspeed allows the model to better account for daily weather variations that impact ridership every hour rather than just on a few specific dates.